In [34]:
import torch
import torch.nn as nn
import torchsummary
import thop

In [21]:
class BaseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False)
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        return x

In [27]:
model = BaseNet()
input_size = (3,256,256)
torchsummary.summary(model, input_size=input_size, batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 16, 256, 256]             432
            Conv2d-2          [1, 32, 256, 256]           4,608
Total params: 5,040
Trainable params: 5,040
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.75
Forward/backward pass size (MB): 24.00
Params size (MB): 0.02
Estimated Total Size (MB): 24.77
----------------------------------------------------------------


In [37]:
class BaseNet(nn.Module):
    def __init__(self, in_ch, out_ch, group):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_ch, out_channels=out_ch,
                               groups=group, kernel_size=3, stride=1,
                               padding=1, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        # x = self.conv2(x)
        return x

class BaseNet_DWCONV(nn.Module):
    def __init__(self, in_ch, out_ch, group):
        super().__init__()
        self.dwconv = nn.Conv2d(in_channels=in_ch, out_channels=in_ch,
                                groups=in_ch, kernel_size=3,
                                stride=1, padding=1, bias=False)

        self.poconv = nn.Conv2d(in_channels=in_ch, out_channels=out_ch,
                                kernel_size=1, stride=1, bias=False)

    def forward(self, x):
        print(x.shape)
        x = self.dwconv(x)
        x = self.poconv(x)

        return x

#### 常规卷积

In [46]:
model = BaseNet(3, 9, 1)
    # print(model)

input_size = (3, 256, 256)
torchsummary.summary(model, input_size=input_size, batch_size=1)
print(model.conv1.weight.data.size())

input_tensor = torch.rand(1,3,256,256)

flops, params = thop.profile(model=model, inputs=(input_tensor,))
print(flops, params)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [1, 9, 256, 256]             243
Total params: 243
Trainable params: 243
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.75
Forward/backward pass size (MB): 4.50
Params size (MB): 0.00
Estimated Total Size (MB): 5.25
----------------------------------------------------------------
torch.Size([9, 3, 3, 3])
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[WARN] Cannot find rule for <class '__main__.BaseNet'>. Treat it as zero Macs and zero Params.
15925248.0 243.0


#### 分组卷积

In [32]:
model = BaseNet(3, 9, 3)
    # print(model)

input_size = (3, 256, 256)
torchsummary.summary(model, input_size=input_size, batch_size=1)
print(model.conv1.weight.data.size())

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [1, 9, 256, 256]              81
Total params: 81
Trainable params: 81
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.75
Forward/backward pass size (MB): 4.50
Params size (MB): 0.00
Estimated Total Size (MB): 5.25
----------------------------------------------------------------
torch.Size([9, 1, 3, 3])


#### 深度可分离卷积

In [33]:
model = BaseNet_DWCONV(3, 9, 3)
torchsummary.summary(model, input_size=input_size, batch_size=1)
print(model.dwconv.weight.data.size())

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [1, 3, 256, 256]              27
            Conv2d-2           [1, 9, 256, 256]              27
Total params: 54
Trainable params: 54
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.75
Forward/backward pass size (MB): 6.00
Params size (MB): 0.00
Estimated Total Size (MB): 6.75
----------------------------------------------------------------
torch.Size([3, 1, 3, 3])


15925248

In [64]:
class BaseNet(nn.Module):
    def __init__(self, in_ch, out_ch, group=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_ch, out_channels=out_ch,
                               groups=group, kernel_size=3, stride=1,
                               padding=1, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        return x

In [67]:
model = BaseNet(6, 3,3)
input_tensor = torch.rand(1,6,64,64)
input_size = tuple(input_tensor.shape[1:])
torchsummary.summary(model, input_size=input_size, batch_size=1)

print("thop:")
flops, params = thop.profile(model=model, inputs=(input_tensor,))
print(flops, params)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [1, 3, 64, 64]              54
Total params: 54
Trainable params: 54
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.09
Forward/backward pass size (MB): 0.09
Params size (MB): 0.00
Estimated Total Size (MB): 0.19
----------------------------------------------------------------
thop:
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[WARN] Cannot find rule for <class '__main__.BaseNet'>. Treat it as zero Macs and zero Params.
221184.0 54.0


In [68]:
model = BaseNet(3, 4)
input_tensor = torch.rand(1,3,64,64)
input_size = tuple(input_tensor.shape[1:])
torchsummary.summary(model, input_size=input_size, batch_size=1)

print("thop:")
flops, params = thop.profile(model=model, inputs=(input_tensor,))
print(flops, params)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [1, 4, 64, 64]             108
Total params: 108
Trainable params: 108
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 0.12
Params size (MB): 0.00
Estimated Total Size (MB): 0.17
----------------------------------------------------------------
thop:
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[WARN] Cannot find rule for <class '__main__.BaseNet'>. Treat it as zero Macs and zero Params.
442368.0 108.0


In [66]:
3*3*64*64*3*6

663552

In [71]:
class BaseNet_DWCONV_sp(nn.Module):
    def __init__(self, in_ch, out_ch, group):
        super().__init__()
        # depthwise
        self.dwconv = nn.Conv2d(in_channels=in_ch, out_channels=in_ch,
                                groups=in_ch, kernel_size=3,
                                stride=1, padding=1, bias=False)
        # pointwise
        self.poconv = nn.Conv2d(in_channels=in_ch, out_channels=out_ch,
                                kernel_size=1, stride=1, bias=False)

    def forward(self, x):
        x = self.dwconv(x)
        x = self.poconv(x)

        return x

model = BaseNet_DWCONV_sp(6, 3, 3)
input_tensor = torch.rand(1,6,64,64)
input_size = tuple(input_tensor.shape[1:])
torchsummary.summary(model, input_size=input_size, batch_size=1)

print("thop:")
flops, params = thop.profile(model=model, inputs=(input_tensor,))
print(flops, params)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [1, 6, 64, 64]              54
            Conv2d-2             [1, 3, 64, 64]              18
Total params: 72
Trainable params: 72
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.09
Forward/backward pass size (MB): 0.28
Params size (MB): 0.00
Estimated Total Size (MB): 0.38
----------------------------------------------------------------
thop:
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[WARN] Cannot find rule for <class '__main__.BaseNet_DWCONV_sp'>. Treat it as zero Macs and zero Params.
294912.0 72.0
